In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
import random
import pickle
from sklearn.preprocessing import StandardScaler
import os
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

2023-06-14 17:44:07.450909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 17:44:07.626096: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 17:44:07.630024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2023-06-14 17:44:07.630049: I tensorflow/compiler/xla/stream_executor/cuda/cudart

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def get_targets_with_weights(batch_data, initial_ensembles, size_ens): 
    
    target_dim = 1
    
    # weights_ann_1 = ann.get_weights()
    
    # h1  = ann.layers[1].output.shape[-1]

    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1
    
    final_output_1 = final_output_1[:,:, 0]
    
    # print(final_output_1.shape, initial_ensembles.shape)
    
    stack = np.hstack((final_output_1, initial_ensembles))

    
    return final_output_1, stack

In [4]:
def ann(hidden = 32, input_shape = 256, output_shape = 1): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(output_shape, activation = "relu")
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [5]:
def generate_initial_ensembles(num_weights, lambda1, size_ens):
    mean_vec = np.zeros((num_weights,))
    cov_matrix = lambda1*np.identity(num_weights)
    mvn_samp = mvn(mean_vec, cov_matrix)
    return mvn_samp.rvs(size_ens)

In [6]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [7]:
samp_ann =  ann(hidden = 16, input_shape = 32, output_shape = 1)

2023-06-13 23:08:46.403236: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2023-06-13 23:08:46.403256: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-13 23:08:46.403275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2518.crane.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2023-06-13 23:08:46.403943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
weights_ann_1 = samp_ann.get_weights()

In [9]:
h1  = samp_ann.layers[1].output.shape[-1]

In [10]:
samp_ann.count_params()

545

In [11]:
hidden_neurons = h1

In [12]:
samp_ann_params = samp_ann.count_params()

In [13]:
# initial_ensembles_for_weights = generate_initial_ensembles(4, 1, 250)
# initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)

In [14]:
# softmax_weights = tf.math.softmax(initial_ensembles_for_weights).numpy()

In [15]:
# softmax_weights.shape

In [16]:
# softmax_weights.shape

In [17]:
# softmax_weights[:2, :, :]

In [18]:
# softmax_weights[:,:2].sum(1)

In [19]:
# softmax_weights[:2,:,:2].sum(-1).reshape(-1,1)

In [20]:
# softmax_weights[:2,:,2:].sum(-1).reshape(-1,1)

In [21]:
# model_1 = softmax_weights[:,:2].sum(1).reshape(-1,1)
    
# model_2 = softmax_weights[:,2:].sum(1).reshape(-1,1)

In [22]:
# model_1 + model_2

In [23]:
# initial_ensembles_for_weights

In [24]:
def get_initial_X_t(data1, data2, size_ens, var_weights = 1.0, var_weight_weights = 4.0):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    
    initial_ensembles1 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data1_out1, data1_stack1 = get_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens)
    
    initial_ensembles2 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data1_out2, data1_stack2 = get_targets_with_weights(data1, initial_ensembles2, size_ens = size_ens)
    
    initial_ensembles3 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data2_out1, data2_stack1 = get_targets_with_weights(data2, initial_ensembles3, size_ens = size_ens)
    
    initial_ensembles4 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data2_out2, data2_stack2 = get_targets_with_weights(data2, initial_ensembles4, size_ens = size_ens)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles_for_weights = generate_initial_ensembles(4, var_weight_weights, size_ens)
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    initial_ensembles_for_L = generate_initial_ensembles(4, var_weights, size_ens)
    initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)    
    
    initial_ensembles_for_D1 = generate_initial_ensembles(1, var_weights, size_ens).reshape(-1,1)
    initial_ensembles_for_D2 = generate_initial_ensembles(1, var_weights, size_ens).reshape(-1,1)
    
    initial_ensembles_for_D1_zero = np.zeros((size_ens,1,1)).reshape(-1,1)
    initial_ensembles_for_D2_zero = np.zeros((size_ens,1,1)).reshape(-1,1)
    
    initial_ensembles_for_D = np.concatenate((np.expand_dims(initial_ensembles_for_D1,1),
                                                       np.expand_dims(initial_ensembles_for_D1_zero,1), 
                                                      np.expand_dims(initial_ensembles_for_D2,1),
                                                       np.expand_dims(initial_ensembles_for_D2_zero,1)), axis = 2)
    
    # print(X_t.shape, initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D), axis = 1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4))
    
    return X_t, initial_ensembles, initial_ensembles_for_weights[:,0,:], initial_ensembles_for_L[:,0,:], initial_ensembles_for_D[:,0,:]

In [25]:
def get_weighted_targets_with_weights(batch_data, initial_ensembles, size_ens, weights): 
    
    target_dim = 1
    
    # weights_ann_1 = ann.get_weights()
    
    # h1  = ann.layers[1].output.shape[-1]

    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1
    
    final_output_1 = final_output_1[:,:, 0]
    
    final_output_1 = final_output_1*weights
    
    # print(final_output_1.shape, initial_ensembles.shape)
    
    stack = np.hstack((final_output_1, initial_ensembles))

    
    return final_output_1, stack

In [26]:
std_targets = pickle.load( open('..//Data//target_scaler.pkl', 'rb'))

In [27]:
# R_t = np.array([[0.02, 0], [0, 0.02]])

In [28]:
# var1 = R_t[0,0]
# var2 = R_t[1,1]
# cov = R_t[1,0]

In [29]:
from scipy.stats import beta

In [30]:
fudging_beta = beta(1,9)

In [31]:
def forward_operation(data1, data2, combined_ensembles , size_ens, fudging_beta):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    params = samp_ann_params
    initial_ensembles1 = combined_ensembles[:, :params]
    initial_ensembles2 = combined_ensembles[:, params:(2*params)]
    initial_ensembles3 = combined_ensembles[:, (2*params):(3*params)]
    initial_ensembles4 = combined_ensembles[:, (3*params):(4*params)]

    
    initial_ensembles_for_weights = combined_ensembles[:, (4*params):(4*params + 4)]
    
    initial_ensembles_for_L = combined_ensembles[:, (4*params + 4):(4*params + 4 + 4)]
    
    initial_ensembles_for_D = combined_ensembles[:,(4*params + 4 + 4):(4*params + 4 + 4 + 4)]
    
    
    softmax_weights = tf.math.softmax(initial_ensembles_for_weights).numpy()
    
    model_1 = softmax_weights[:, :2].sum(1).reshape(-1,1) +  fudging_beta.rvs(size_ens).reshape(-1,1)
    
    # model_1 = np.min(model_1 -fudging_factor)
    
    model_2 = softmax_weights[:, 2:].sum(1).reshape(-1,1) +  fudging_beta.rvs(size_ens).reshape(-1,1)
    
    
    model_1_plus_model_2 = model_1 + model_2
    
    model_1 = model_1/model_1_plus_model_2
    
    model_2 = model_2/model_1_plus_model_2
    
    
    # print(np.mean(model_1 + model_2))
    
    data1_out1, data1_stack1 = get_weighted_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens,
                                                                  weights=model_1)
    
    data1_out2, data1_stack2 = get_weighted_targets_with_weights(data1, initial_ensembles2, size_ens = size_ens,
                                                                weights=model_1)
    
    data2_out1, data2_stack1 = get_weighted_targets_with_weights(data2, initial_ensembles3, size_ens = size_ens,
                                                                 weights=model_2)
    
    data2_out2, data2_stack2 = get_weighted_targets_with_weights(data2, initial_ensembles4, size_ens = size_ens,
                                                                  weights=model_2)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4, 
                        initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D))
    
    # print(X_t.shape)
    
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)
    
    initial_ensembles_for_D = np.expand_dims(initial_ensembles_for_D,1)
    
    # print(initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D), axis = 1)
    
    weighted_alogp = data1_out1 + data2_out1
    
    weighted_psa = data1_out2 + data2_out2
    
    return X_t, initial_ensembles, weighted_alogp, weighted_psa, model_1, model_2

In [32]:
def forward_operation_test(data1, data2, combined_ensembles , size_ens):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    params = samp_ann_params
    initial_ensembles1 = combined_ensembles[:, :params]
    initial_ensembles2 = combined_ensembles[:, params:(2*params)]
    initial_ensembles3 = combined_ensembles[:, (2*params):(3*params)]
    initial_ensembles4 = combined_ensembles[:, (3*params):(4*params)]

    
    initial_ensembles_for_weights = combined_ensembles[:, (4*params):(4*params + 4)]
    
    initial_ensembles_for_L = combined_ensembles[:, (4*params + 4):(4*params + 4 + 4)]
    
    initial_ensembles_for_D = combined_ensembles[:,(4*params + 4 + 4):(4*params + 4 + 4 + 4)]
    
    
    softmax_weights = tf.math.softmax(initial_ensembles_for_weights).numpy()
    
    model_1 = softmax_weights[:, :2].sum(1).reshape(-1,1) 
    
    # model_1 = np.min(model_1 -fudging_factor)
    
    model_2 = softmax_weights[:, 2:].sum(1).reshape(-1,1) 
    
    
#     model_1_plus_model_2 = model_1 + model_2
    
#     model_1 = model_1/model_1_plus_model_2
    
#     model_2 = model_2/model_1_plus_model_2
    
    
    # print(np.mean(model_1 + model_2))
    
    data1_out1, data1_stack1 = get_weighted_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens,
                                                                  weights=model_1)
    
    data1_out2, data1_stack2 = get_weighted_targets_with_weights(data1, initial_ensembles2, size_ens = size_ens,
                                                                weights=model_1)
    
    data2_out1, data2_stack1 = get_weighted_targets_with_weights(data2, initial_ensembles3, size_ens = size_ens,
                                                                 weights=model_2)
    
    data2_out2, data2_stack2 = get_weighted_targets_with_weights(data2, initial_ensembles4, size_ens = size_ens,
                                                                  weights=model_2)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4, 
                        initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D))
    
    # print(X_t.shape)
    
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)
    
    initial_ensembles_for_D = np.expand_dims(initial_ensembles_for_D,1)
    
    # print(initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D), axis = 1)
    
    weighted_alogp = data1_out1 + data2_out1
    
    weighted_psa = data1_out2 + data2_out2
    
    return X_t, initial_ensembles, weighted_alogp, weighted_psa, model_1, model_2

In [33]:
# samp_ann =  ann(hidden = 16, input_shape = 32, output_shape = 1)

In [34]:
total_weights = 4*(samp_ann.count_params() + 1 + 1 + 1)

In [35]:
reduction = 10

In [36]:
size_ens = total_weights//reduction

In [37]:
size_ens

219

In [38]:
G_t = [[1, 0, 1, 0], [0, 1, 0, 1]]
G_t = np.array(G_t).T

In [39]:
# G_t

In [40]:
def get_predictions(data1, data2, initial_ensembles, fudging_beta  =fudging_beta): 
    _,_, weighted_alogp, weighted_psa, w1, w2 = forward_operation(data1, data2, initial_ensembles, size_ens = size_ens, fudging_beta = fudging_beta)
    weighted_alogp = np.expand_dims(weighted_alogp,-1)
    weighted_psa = np.expand_dims(weighted_psa,-1)
    preds = np.concatenate((weighted_alogp, weighted_psa),-1)
    return preds, w1, w2

In [41]:
def get_predictions_test(data1, data2, initial_ensembles): 
    _,_, weighted_alogp, weighted_psa, w1, w2 = forward_operation_test(data1, data2, initial_ensembles, size_ens = size_ens)
    weighted_alogp = np.expand_dims(weighted_alogp,-1)
    weighted_psa = np.expand_dims(weighted_psa,-1)
    preds = np.concatenate((weighted_alogp, weighted_psa),-1)
    return preds, w1, w2

In [42]:
def calculate_mu_bar_G_bar(data1, data2, initial_ensembles, fudging_beta):
    H_t = np.hstack((np.identity(data1.shape[0]), np.zeros((data1.shape[0], samp_ann_params + 1 + 1 + 1))))
    mu_bar = initial_ensembles.mean(0)
    X_t,_, _, _, _, _ = forward_operation(data1, data2, initial_ensembles, size_ens = size_ens, fudging_beta = fudging_beta)
    X_t = X_t.transpose((0,2,1))
    X_t = X_t.reshape(X_t.shape[0], X_t.shape[1]*X_t.shape[2])
    script_H_t = np.kron(G_t.T, H_t)
    G_u = (script_H_t@X_t.T)
    G_u = G_u.T
    # weighted_alogp = np.expand_dims(weighted_alogp,-1)
    # weighted_psa = np.expand_dims(weighted_psa,-1)
    # G_u = np.concatenate((weighted_alogp, weighted_psa), axis = -1)
    # G_u = G_u.transpose((0,2,1))
    # G_u = G_u.reshape(G_u.shape[0], G_u.shape[1]*G_u.shape[2])
    # G_u
    G_bar = (G_u.mean(0)).ravel()
    return mu_bar.reshape(-1,1), G_bar.reshape(-1,1), G_u

In [43]:
def calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u): 
    u_j_minus_u_bar = initial_ensembles - mu_bar.reshape(1,-1)
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    c = np.zeros((total_weights, G_bar.shape[0]))
    for i in range(0, size_ens): 
        c += np.kron(u_j_minus_u_bar[i, :].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return c/size_ens, G_u_minus_G_bar

In [44]:
def calculate_D_u( G_bar, G_u): 
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    d = np.zeros((G_bar.shape[0], G_bar.shape[0]))
    for i in range(0, size_ens): 
        d += np.kron(G_u_minus_G_bar[i,:].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return d/size_ens

In [45]:
def get_updated_ensemble(data1, data2, initial_ensembles, y_train, size_ens = size_ens, inflation_factor = 1.0, fudging_beta = fudging_beta):
    mu_bar, G_bar, G_u = calculate_mu_bar_G_bar(data1, data2, initial_ensembles, fudging_beta)
    C, G_u_minus_G_bar = calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u)
    D = calculate_D_u( G_bar, G_u)
    _, R_t = create_cov(data1.shape[0],initial_ensembles)
    inflation = np.identity(R_t.shape[0])*inflation_factor
    D_plus_cov = D + (R_t *inflation_factor)
    # all_covs = np.array(all_covs)
    # D_plus_cov = D + R_t
    D_plus_cov_inv = np.linalg.inv(D_plus_cov)
    mid_quant = C@D_plus_cov_inv
    noise_vec_mean = np.zeros((R_t.shape[0], ))
    noise_mvn = mvn(noise_vec_mean, R_t)
    fudging = noise_mvn.rvs(size_ens)
    interim = (y_train.T.flatten().reshape(1,-1) + fudging)
    right_quant = interim - G_u
    # print(mid_quant.shape, right_quant.shape)
    mid_times_right = mid_quant@right_quant.T
    updated_ensemble = (initial_ensembles + mid_times_right.T)
    return updated_ensemble

In [46]:
target_dim = 2

In [47]:
lambda_D = 1

In [48]:
def inverse_transform(data, idx):
    data_cur = data[idx, :, :]
    inv_data_cur = std_targets.inverse_transform(data_cur)
    return inv_data_cur

In [49]:
from joblib import Parallel, delayed

In [50]:
def create_cov(shape, initial_ensembles):
    cov_part = initial_ensembles[:, -8:-4]
    cov_part = cov_part.mean(0)
    # variances = tf.math.softplus(cov_part[:2]).numpy()
    variances = cov_part[:2]
    covariances = cov_part[2:]
    base_cov = np.identity(target_dim)
    base_cov[0,0] = variances[0]
    base_cov[1,1] = variances[1]
    base_cov[0,1] = covariances[0]
    base_cov[1,0] = covariances[1]
    
    variances1 = tf.math.softplus(initial_ensembles[:, -4:]).numpy()
    variances1 = variances1.mean(0)
    base_variances = np.identity(target_dim)
    base_variances[0,0] = variances1[0]
    base_variances[1,1] = variances1[2]
    
    final = np.linalg.cholesky(base_cov@base_cov.T + base_variances)
    cov_mat = final@final.T
    cov_mat_final = cov_mat
    # cov_mat_final = cov_mat@cov_mat.T
    
    if is_pos_def(cov_mat_final) != True:
        print("resulting cov matrix is not positive semi definite")
        pass
    
    # print(np.linalg.det(cov_mat_final))
    
    var1 = cov_mat_final[0,0]
    var2 = cov_mat_final[1,1]
    cov = cov_mat_final[1,0]

    n = shape
    
    ul = var1*np.identity(n)
    lr = var2*np.identity(n)
    ur = cov*np.identity(n)
    ll = ur.T    
    
    first_row = np.hstack((ul, ur))
    second_row = np.hstack((ll, lr))
    
    R_t = np.vstack((first_row, second_row))
    
    # R_t = block_diag(*([cov_mat_final] * n))
    
    # R_t = np.linalg.inv(R_t)
    
    return cov_mat_final, R_t
    

In [51]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [52]:
with open("..//Data//smiles_to_rdkit_80_20_with_cov_minus_0.2_var.pickle", "rb") as f: 
    catch = pickle.load(f)

In [53]:
# idx = 1

In [54]:
def prepare_data(idx, var_weights = 1.0, var_weight_weights = 4.0): 
    catch_idx = catch[idx]
    x_train, x_valid, y_train, y_valid = catch_idx[0], catch_idx[1], catch_idx[2], catch_idx[3]
    y_train_actual, y_train = y_train[:,:2], y_train[:,2:]
    y_valid_actual, y_valid = y_valid[:,:2], y_valid[:,2:]
    smiles_feats_train = x_train[:, :32]
    rdkit_feats_train = x_train[:, 32:]
    smiles_feats_valid = x_valid[:, :32]
    rdkit_feats_valid = x_valid[:, 32:]

    X_t, initial_ensembles, initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D = get_initial_X_t(smiles_feats_train, rdkit_feats_train, size_ens = size_ens, var_weights = var_weights, var_weight_weights = var_weight_weights)
    initial_ensembles = np.hstack((initial_ensembles, initial_ensembles_for_weights, initial_ensembles_for_L, initial_ensembles_for_D))
    
    return smiles_feats_train, rdkit_feats_train, smiles_feats_valid, rdkit_feats_valid, y_train, y_train_actual, y_valid, y_valid_actual, initial_ensembles 

In [55]:
# smiles_feats_train, rdkit_feats_train, smiles_feats_valid, rdkit_feats_valid, y_train, y_train_actual, y_valid, y_valid_actual, initial_ensembles  = prepare_data(idx)

In [56]:
# y_train

In [57]:
# y_train_actual

In [58]:
from scipy.linalg import norm

In [59]:
# beta(1,19).rvs(size_ens).mean()

In [60]:
def get_results(idx, var_weights = 1.0, var_weight_weights = 1.0, inflation_factor = 1.2, fudging_beta = beta(1,19)):
    # print('var_weights' + str(var_weights))
    # print('inflation_factor' + str(inflation_factor))
    # print('var_weight_weights' + str(var_weight_weights))
    smiles_feats_train, rdkit_feats_train, smiles_feats_valid, rdkit_feats_valid, y_train, y_train_actual, y_valid, y_valid_actual, initial_ensembles  = prepare_data(idx, var_weights = var_weights, var_weight_weights =var_weight_weights)
    # print(R_t.shape)
    best_train_width_mean = 100000
    
    for i in range(0,10000):
        # print(i)
    
        c = np.zeros((2,2))
        initial_ensembles = get_updated_ensemble(smiles_feats_train, rdkit_feats_train, initial_ensembles, y_train, size_ens, inflation_factor = inflation_factor, fudging_beta = fudging_beta)
        # print(inflation_factor)
        G_u_train, w1, w2 = get_predictions(smiles_feats_train, rdkit_feats_train, initial_ensembles, fudging_beta)

        catch = Parallel(n_jobs = 15, verbose = 0)(delayed(inverse_transform)(G_u_train, i)  for i in range(G_u_train.shape[0]))
        G_u_train = np.array(catch)
    
        y_train_cur = std_targets.inverse_transform(y_train_actual)
    
        li_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[0,:,:]   
        ui_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[1,:,:]
    
        width_train = ui_train - li_train
        avg_width_train = width_train.mean(0)
    
        ind_train = (y_train_cur >= li_train) & (y_train_cur <= ui_train)
        coverage_train= ind_train.mean(0)
    
        averaged_targets_train = G_u_train.mean(0)
        rmse_train = np.sqrt(((y_train_cur -averaged_targets_train)**2).mean(0))
    # print(rmse_train, coverage_train, avg_width_train)
    
        G_u_test, _, _ = get_predictions_test(smiles_feats_valid, rdkit_feats_valid, initial_ensembles)
    
        catch = Parallel(n_jobs = 15, verbose = 0)(delayed(inverse_transform)(G_u_test, i)  for i in range(G_u_test.shape[0]))
        G_u_test = np.array(catch)
    
        y_valid_cur = std_targets.inverse_transform(y_valid_actual)    
    
        li_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[0,:,:]   
        ui_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[1,:,:]
    
        width_test = ui_test - li_test
        avg_width_test = width_test.mean(0)
    
        ind_test = (y_valid_cur >= li_test) & (y_valid_cur <= ui_test)
        coverage_test= ind_test.mean(0)
    
        averaged_targets_test = G_u_test.mean(0)
        rmse_test = np.sqrt(((y_valid_cur -averaged_targets_test)**2).mean(0))    
    
        # weight_norms = np.array(norm(initial_ensembles, ord = 2, axis = 1))
        # weight_norm_mean.append(weight_norms.mean())
        # weight_norm_sd.append(weight_norms.std())
    
        cov_mat_final, _ = create_cov(smiles_feats_train.shape[0],initial_ensembles)
        
        # print("standardized_scale_R_t")
        # print(np.diag(cov_mat_final), cov_mat_final[0,1])
        
        # print(w1.shape)
        
        li_smiles_weight = np.percentile(w1, axis = 0, q = (2.5, 97.5))[0][0]
        
        # print(np.percentile(w1, axis = 0, q = (2.5, 97.5)))
        
        ui_smiles_weight = np.percentile(w1, axis = 0, q = (2.5, 97.5))[1][0]      
        
        # print(coverage_train.tolist(), avg_width_train.tolist(), rmse_train.tolist())
        # print(coverage_test.tolist(), avg_width_test.tolist(), rmse_test.tolist())
        # print(w1.mean())
        # print(li_smiles_weight, ui_smiles_weight)
        # print(avg_width_train.tolist(), coverage_train.tolist(), rmse_train.tolist(), avg_width_test.tolist(), coverage_test.tolist(), rmse_test.tolist(), w1.mean())

        if (avg_width_train.mean() < best_train_width_mean) & (coverage_train.mean() > 0.95): 
            # print("went here")
            best_train_width_mean = avg_width_train.mean()
            best_train_width = avg_width_train
            best_smiles_weight = w1.mean()
            best_coverage_train = coverage_train
            best_rmse_train = rmse_train
        
            best_test_width = avg_width_test

            best_coverage_test = coverage_test    
            best_rmse_test = rmse_test
            
            best_li_smiles_weight = li_smiles_weight
            
            best_ui_smiles_weight = ui_smiles_weight
    
        if coverage_train.mean() < 0.95:
            
            # print()
            # print(best_train_width.tolist(), best_coverage_train.tolist(), best_rmse_train.tolist(), best_test_width.tolist(), best_coverage_test.tolist(), best_rmse_test.tolist(), best_smiles_weight, flush = True)
            print("done for fold" + str(idx), flush = True)
            print("train_width" + str(best_train_width.tolist()), flush = True)
            print("test_width" + str(best_test_width.tolist()), flush = True)
            print("smiles_weight" + str(best_smiles_weight), flush = True)
            print("rmse_train" + str(best_rmse_train.tolist()), flush = True)
            print("rmse_test" + str(best_rmse_test.tolist()), flush = True)
            print("smiles_weight_ci" + str([best_li_smiles_weight, best_ui_smiles_weight]), flush = True)
            
            return [best_train_width.tolist(), best_coverage_train.tolist(), best_rmse_train.tolist(), best_test_width.tolist(), best_coverage_test.tolist(), best_rmse_test.tolist(), best_smiles_weight, best_li_smiles_weight, best_ui_smiles_weight]


In [61]:
# idx

In [62]:
%%time
# get_results(43, var_weights = 1.0, var_weight_weights = 3.0, inflation_factor =1.0)

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 14.3 µs


In [63]:
# from joblib import Parallel, delayed

In [64]:
catch_all = Parallel(n_jobs = 15, verbose = 10)(delayed(get_results)(idx,var_weights = 1.0, var_weight_weights = 3.0, inflation_factor =1.0) for idx in range(0,50))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.


var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold8
train_width[1.0847291789892317, 19.20892224040216]
test_width[0.8361406117550596, 16.483142109034002]
smiles_weight0.8414032994999122
rmse_train[0.24552270613384744, 4.530572365940998]
rmse_test[0.21718087658023108, 4.677358487734904]
smiles_weight_ci[0.7401994465371133, 0.9078455549069083]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold11
train_width[0.967993272210303, 20.704152472140464]
test_width[0.7482144231519666, 17.295827605570956]
smiles_weight0.8542654665657387
rmse_train[0.1979292069163449, 4.7662727307101065]
rmse_test[0.22212147089763598, 4.728060661659346]
smiles_weight_ci[0.7556661641957848, 0.91772071397014]


[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:  5.0min


var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold1
train_width[0.828372183024853, 17.13720592220191]
test_width[0.6966773199338674, 16.342220673399265]
smiles_weight0.7673864004496154
rmse_train[0.19691802196864908, 4.6654230923707045]
rmse_test[0.1683771308280604, 5.335614353009048]
smiles_weight_ci[0.680958972813426, 0.8344938608343536]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold12
train_width[0.9334365849378621, 17.790361271937268]
test_width[0.6358937089643601, 14.367446186958931]
smiles_weight0.8528459402104711
rmse_train[0.2221822307810798, 4.2904242403861765]
rmse_test[0.18973549970058276, 4.458247533418008]
smiles_weight_ci[0.7511302651582746, 0.9076813203091443]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold2
train_width[0.7649075432628195, 17.584666724634012]
test_width[0.7464124450295009, 17.805106729272946]
smiles_weight0.7655850358770828
rmse_train[0.1613522740793943, 4.15318430910195]
rmse_test[0.5280

[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed:  6.4min


...

[Parallel(n_jobs=15)]: Done  39 out of  50 | elapsed: 17.7min remaining:  5.0min


var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold44
train_width[0.9020314782180112, 20.18042846233878]
test_width[0.7193264281449496, 17.446271289530998]
smiles_weight0.7868127864749257
rmse_train[0.21006811581957602, 5.152065860362461]
rmse_test[0.16145169773372134, 4.517223163082472]
smiles_weight_ci[0.6975669902407126, 0.8493017309137704]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold36
train_width[0.8821038529278802, 10.782495003743573]
test_width[0.46821620149427823, 8.49275182463412]
smiles_weight0.8597125405452083
rmse_train[0.14809822615294727, 3.110678566230732]
rmse_test[0.22319082415252076, 3.0385337742166505]
smiles_weight_ci[0.7628225597653066, 0.912699776550275]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold43
train_width[0.9090653978375145, 16.893003819344713]
test_width[0.8050917070838822, 16.231096639718004]
smiles_weight0.7915711542559956
rmse_train[0.21169148695831033, 4.164443707671385]
rmse_test[0

[Parallel(n_jobs=15)]: Done  45 out of  50 | elapsed: 19.0min remaining:  2.1min


var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold46
train_width[0.9551689694646703, 21.035760670295222]
test_width[0.9600954727237968, 21.544872129340018]
smiles_weight0.6884467967216282
rmse_train[0.21304763759993764, 4.383174611609016]
rmse_test[0.2647954123456012, 4.174520424506186]
smiles_weight_ci[0.599303955808443, 0.7684889835785843]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold47
train_width[0.8761950675297266, 20.57735717257833]
test_width[0.7345166508107792, 18.251628691668817]
smiles_weight0.807574912349228
rmse_train[0.21733673826712813, 4.1571765840676935]
rmse_test[0.2339842125616461, 3.8100281071012416]
smiles_weight_ci[0.7144078553977549, 0.8692198496754077]
var_weights1.0
inflation_factor1.0
var_weight_weights3.0
done for fold45
train_width[0.7337318165461738, 12.735470063709466]
test_width[0.5218110610857153, 10.687959619571645]
smiles_weight0.8364981681876309
rmse_train[0.13966961558593874, 3.232566553292188]
rmse_test[0.

[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed: 20.6min finished


In [65]:
# item

In [66]:
# catch_all[-1]

In [67]:
# catch_inner

In [68]:
all_catch = []
for item in catch_all:
    catch_inner = []
    for inner in item:
        if type(inner) == list:
            for inner1 in inner:
                catch_inner.append(inner1)
        if type(inner) != list:
            catch_inner.append(inner)
    all_catch.append(catch_inner)

In [69]:
results_df = pd.DataFrame(all_catch)

In [70]:
results_df.shape

(50, 15)

In [71]:
# results_df

In [72]:
# results_df.iloc[:,-1].mean()

In [76]:
col_names = ["Alop_Train_Width", "PSA_Train_Width", "Alop_Train_Coverage", "PSA_Train_Coverage", 
            "Alop_Train_RMSE", "PSA_Train_RMSE", "Alop_Test_Width", "PSA_Test_Width", "Alop_Test_Coverage", "PSA_Test_Coverage", 
            "Alop_Test_RMSE", "PSA_Test_RMSE", "Smiles_Avg_Weight", "Lower_Interval_Smiles_Weight", "Upper_Interval_Smiles_Weight"]

In [74]:
# results_df.head()

In [77]:
results_df.columns = col_names

In [78]:
results_df["indicator"] = (results_df["Lower_Interval_Smiles_Weight"].values < 0.80) & (results_df["Upper_Interval_Smiles_Weight"].values >= 0.80)

In [79]:
np.mean(results_df["indicator"])

0.96

In [11]:
# results_df = pd.read_csv("..//Data//smiles_rdkit_80_20__with_cov_minus_0.2_Simulation_added_beat_noise.csv")

In [5]:
results_df["width_weight_CI"] = results_df["Upper_Interval_Smiles_Weight"].values - results_df["Lower_Interval_Smiles_Weight"].values

In [10]:
# results_df.head()

,Alop_Train_Width,PSA_Train_Width,Alop_Train_Coverage,PSA_Train_Coverage,Alop_Train_RMSE,PSA_Train_RMSE,Alop_Test_Width,PSA_Test_Width,Alop_Test_Coverage,PSA_Test_Coverage,Alop_Test_RMSE,PSA_Test_RMSE,Smiles_Avg_Weight,Lower_Interval_Smiles_Weight,Upper_Interval_Smiles_Weight,indicator,width_weight_CI
0,0.888638,15.866021,0.966620,0.942976,0.182282,4.157536,0.586512,12.277395,0.970833,0.933333,0.167477,3.673361,0.851444,0.763130,0.911440,True,0.148309
1,0.828372,17.137206,0.965229,0.952712,0.196918,4.665423,0.696677,16.342221,0.920833,0.941667,0.168377,5.335614,0.767386,0.680959,0.834494,True,0.153535
2,0.764908,17.584667,0.962448,0.966620,0.161352,4.153184,0.746412,17.805107,0.950000,0.966667,0.528050,5.054569,0.765585,0.671587,0.835498,True,0.163911
3,0.724796,14.029597,0.954103,0.952712,0.138197,3.465563,0.515921,11.487300,0.904167,0.975000,0.210036,5.375835,0.860179,0.762245,0.910718,True,0.148473
4,0.673729,12.861163,0.959666,0.956885,0.172656,3.168526,0.626906,12.570534,0.862500,0.908333,0.491792,10.107197,0.824025,0.734312,0.883947,True,0.149635


In [7]:
results_df.mean().reset_index()

,index,0
0,Alop_Train_Width,0.825203
1,PSA_Train_Width,16.096670
2,Alop_Train_Coverage,0.962698
3,PSA_Train_Coverage,0.952045
4,Alop_Train_RMSE,0.178951
5,PSA_Train_RMSE,3.954676
6,Alop_Test_Width,0.622456
7,PSA_Test_Width,13.660121
8,Alop_Test_Coverage,0.919500
9,PSA_Test_Coverage,0.941250


In [8]:
results_df.to_csv("..//Data//smiles_rdkit_80_20__with_cov_minus_0.2_Simulation_added_beat_noise.csv", index = False)

In [ ]:
# results_df.mean().reset_index()

In [ ]:
# results_df.std()